# Section 1: How to deal with Missing Data

Missing data are defined as values that are not available and that would be meaningful if they are observed. Missing data can be anything from missing sequence, incomplete feature, files missing, information incomplete, data entry error etc. Most datasets in the real world contain missing data. Before you can use data with missing data fields, you need to transform those fields so they can be used for analysis and modelling. 

Like many other aspects of data science, this too may actually be more art than science. Understanding the data and the domain from which it comes is very important.


**For a better understanding, let's import the *train file* that we filtered on step 1. Remember that we already discarded the variables that had a huge amount of missing data.**

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

In [3]:
# Import the train, so that the data that we are going to use to build / train the model
train_path = r'./input/Train_Filtered.csv'
train = pd.read_csv(train_path)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 77 columns):
Unnamed: 0       1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non

#### Let's execute the following code to see the percentage of missing values of each of the variables we have

In [4]:
# We will define a a function to map the amount and % of missing values for the different variables

def show_null(df):
    null_columns = (df.isnull().sum(axis = 0)/len(df)).sort_values(ascending=False).index
    null_data = pd.concat([df.isnull().sum(axis = 0),
                           (df.isnull().sum(axis = 0)/len(df)).sort_values(ascending=False),
                           df.loc[:, df.columns.isin(list(null_columns))].dtypes]
                          , axis=1, sort= 'True')
    null_data = null_data.rename(columns={0: 'Amount', 
                                          1: 'Missing data (%)', 
                                          2: 'type'}).sort_values(ascending=False, by = 'Missing data (%)')
    null_data = null_data[null_data["Amount"]!=0]
    return null_data

show_null(train)

,Amount,Missing data (%),type
FireplaceQu,690,0.472603,object
LotFrontage,259,0.177397,float64
GarageCond,81,0.055479,object
GarageFinish,81,0.055479,object
GarageQual,81,0.055479,object
GarageType,81,0.055479,object
GarageYrBlt,81,0.055479,float64
BsmtExposure,38,0.026027,object
BsmtFinType2,38,0.026027,object
BsmtQual,37,0.025342,object


Having a look at the results below, it is possible to describe two types of variables: **numerical (float64)** and **categorical (object)**.
Let's see how can we replace this missing values for each type of variable

**a) Numerical variables**

We have three variables which are numeric: *'LotFrontage'*, *'GarageYrBlt'* and *'MasVnrArea'*. 

One possible solution could be to **drop the rows (Houses) that contain missing values**. In that case, as the percentage of that type of values is relativelly small, the easiest solution is to **replace those NaN by zero**. In principle, there will not have a big impact on the model performace.

In [5]:
# We will select the numerical variables and replace them with zero

for col in ('LotFrontage', 'GarageYrBlt', 'MasVnrArea'):
    train[col] = train[col].fillna(0)

**b) Categorical variables**

A categorical variable is one that has two or more categories.

Having a look at the data that we are analysing it is possible to observe that the variable *Street* contains two categories: Pave and Grvl which is related to the condition of the street where the house is located.

In [95]:
print('Categories for Street variable:')
print(train['Street'].unique())

Categories for Street variable:
['Pave' 'Grvl']


On the other hand, if we have a look at the variable *MSZoning* it is posible to detect five categories: C (all), FV, RH, RL and RM

In [11]:
print('Categories for MSZoning variable:' )
print(train['MSZoning'].unique()) 

Categories for MSZoning variable:
['RL' 'RM' 'C (all)' 'FV' 'RH']


Since these types of variables contain information in the form of characters and we do not know this information, the most commonly used option is to replace these values with None. We will use the following code, which is a really simple one.

In [12]:
for i in train:
    train[i] = train[i].fillna('None') # we replace NaN's by 'None'

Next step is to check whether , by calling the function that we defined above (*show_null*)

In [8]:
show_null(train)

,Amount,Missing data (%),type


As we can see, we manage to fill out all the missing values present in the dataset 

# Section 2: How to deal with Missing Data

We are going to imagine that once we have selected the most important variables to predict the price of housing, we have found categorical variables. As it has been commented, as the predictive models are based on mathematics in order to build them, it is necessary to have "numbers". In this section of the Appendix, **the translation of categorical variables (characters) into numbers or vectors will be described.**

This translation of the variables is called **encoding.**

For a better understanding, let's select three variables: *'SalePrice'*, *'Street'*, and *'MSZoning'*.

In [23]:
train_ = train[['SalePrice', 'Street', 'MSZoning']]

Basically, there are two types of categorical data types in statistics:

* *Nominal*: Nominal values which represent discrete units.For example Street variable has Pave and Grvl categorical values. So here, we have used One Hot Encoding.

* *Ordinal*: ordinal values represent discrete and ordered units. It is therefore nearly the same as nominal data, except that it’s ordering matters. That's where label encoder comes into play. For example : MSZoning feature, it may be in 'RL' 'RM','FV','RH'.

How to deal with them? Let's find out in the following lines

**a)Nominal variables**

A common approach is called **one hot encoding**. Despite the different names, the basic strategy is to convert each category value into a new column and assigns a 1 or 0 (True/False) value to the column. This has the benefit of not weighting a value improperly but does have the downside of adding more columns to the data set.

Pandas supports this feature using *get_dummies*. This function is named this way because it creates dummy/indicator variables ( 1 or 0).

In [24]:
train_ = pd.get_dummies(train_, columns=["Street"])
print(train_)

      SalePrice MSZoning  Street_Grvl  Street_Pave
0        208500       RL            0            1
1        181500       RL            0            1
2        223500       RL            0            1
3        140000       RL            0            1
4        250000       RL            0            1
...         ...      ...          ...          ...
1455     175000       RL            0            1
1456     210000       RL            0            1
1457     266500       RL            0            1
1458     142125       RL            0            1
1459     147500       RL            0            1

[1460 rows x 4 columns]


In [27]:
train_.groupby(['Street_Grvl', 'Street_Pave']).count()

,,SalePrice,MSZoning
Street_Grvl,Street_Pave,,
0,1,1454,1454
1,0,6,6


We can see that this function has splitted the Street variable into two new variables: *Street_Grvl* and *Street_Grvl*. 

Each new column generated contains values 0 or 1 for each row (house). In this codification, 1 indicates that the house has this characteristic whereas 0 does not.

**Ordinal variables**

There are a lot of ways to convert this kind of categorical text data into numerical data. In that case, we use one which is quite useful and easy: the *Label Encoder* module from Python. 

**LabelEncoder** encodes labels of a categorical characteristic into numeric values between 0 and the number of classes minus 1. Once instantiated, the *fit* method trains it (creating the mapping between labels and numbers) and the transform method transforms the labels that are included as an argument into the corresponding numbers. The *fit_transform* method performs both actions simultaneously.

In [28]:
from sklearn.preprocessing import LabelEncoder
# Process columns and apply LabelEncoder to categorical features
for i in ['MSZoning']:
    lbl = LabelEncoder() 
    lbl.fit(list(train_[i].values)) 
    train_[i] = lbl.transform(list(train_[i].values))

If we make a table of frequency of this variable we can see that each of the items of the variable have been coded to 0, 1,2,3 or 4 depending on the category to which they belong.

In [29]:
train_.groupby('MSZoning').count()

,SalePrice,Street_Grvl,Street_Pave
MSZoning,,,
0,10,10,10
1,65,65,65
2,16,16,16
3,1151,1151,1151
4,218,218,218


**Now that we have our categorical variables coded in numbers, they can be used to train a prediction model.**